In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from IPython.display import Image
!pip install keras-tuner --upgrade
from keras_tuner import RandomSearch
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom
from tensorflow.keras.layers.experimental.preprocessing import RandomContrast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
! unzip /content/drive/MyDrive/Combined_Dataset.zip
dataset_dir = "/content/content/AIRecognition"

Streaming output truncated to the last 5000 lines.
  inflating: content/AIRecognition/fake/fake-v2/13943.png  
  inflating: content/AIRecognition/fake/fake-v2/2587.jpg  
  inflating: content/AIRecognition/fake/fake-v2/11963.jpg  
  inflating: content/AIRecognition/fake/fake-v2/20878.png  
  inflating: content/AIRecognition/fake/fake-v2/13839.png  
  inflating: content/AIRecognition/fake/fake-v2/19956.png  
  inflating: content/AIRecognition/fake/fake-v2/22093.png  
  inflating: content/AIRecognition/fake/fake-v2/2704.jpg  
  inflating: content/AIRecognition/fake/fake-v2/16898.png  
  inflating: content/AIRecognition/fake/fake-v2/20703.png  
  inflating: content/AIRecognition/fake/fake-v2/23931.png  
  inflating: content/AIRecognition/fake/fake-v2/17039.png  
  inflating: content/AIRecognition/fake/fake-v2/2647.jpg  
  inflating: content/AIRecognition/fake/fake-v2/24691.png  
  inflating: content/AIRecognition/fake/fake-v2/19245.png  
  inflating: content/AIRecognition/fake/fake-v2/1372

In [ ]:
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),  # Randomly flip images horizontally and vertically
    RandomRotation(0.2),  # Randomly rotate images by 20%
    RandomZoom(0.2),  # Randomly zoom into images
    RandomContrast(0.2),  # Randomly adjust contrast
    # normalization_layer
])

In [ ]:
img_height = 224
img_width = 224
batch_size = 100

# Load the training data
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir,
  validation_split=0.2,
  shuffle=True,
  subset="training",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

# Load the validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir,
  validation_split=0.2,
  subset="validation",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

class_names = train_ds.class_names
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

Found 27548 files belonging to 2 classes.
Using 22039 files for training.
Found 27548 files belonging to 2 classes.
Using 5509 files for validation.


In [ ]:
# Building the Transfer Learning model using EfficientNetV2B0
def build_model(hp):
  EfficientNet_base_model = tf.keras.applications.EfficientNetV2B0(
      include_top = False,
      weights = 'imagenet',
      input_shape = (img_height, img_width, 3),
      pooling = 'max'
  )
  EfficientNet_base_model.trainable = True

  # Create a new model on top of the EfficientNet base
  inputs = tf.keras.Input(shape = (img_height, img_width, 3))
  x = EfficientNet_base_model(inputs, training = False)
  x = BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001)(x)
  x = Dense(hp.Choice('units', values = [64, 128, 256]),
            kernel_regularizer = regularizers.l2(0.01),
            activity_regularizer = regularizers.l1(0.01),
            bias_regularizer = regularizers.l1(0.01),
            activation = 'relu')(x)
  x = Dropout(rate = hp.Choice('dropout', values=[0.3, 0.4, 0.5]), seed = 512)(x)
  x = Dense(64, activation = 'relu')(x)
  outputs = Dense(1, activation = 'sigmoid')(x)
  EfficientNet_model = tf.keras.Model(inputs, outputs)

  # Compile the Transfer Learning model
  EfficientNet_model.compile(
      optimizer = tf.keras.optimizers.Adamax(learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
      loss = tf.keras.losses.BinaryCrossentropy(),
      metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
  )

  return EfficientNet_model
# Build the Transfer Learning model so we can see a summary
# EfficientNet_model.summary()

In [ ]:
hyperparameters = {
    'units': [64, 128, 256],
    'dropout_rate': [0.3, 0.4, 0.5],
    'learning_rate': [1e-2, 1e-3, 1e-4]
}

In [ ]:
!pip install keras-tuner

from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='/content/model_tuning',
    project_name='model_tuning'
)

tuner.search(train_ds, validation_data=val_ds, epochs=20)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")
best_model = tuner.get_best_models(num_models=1)[0]

best_model.save('best_model')
! scp -r /content/best_model /content/drive/MyDrive/

Trial 5 Complete [01h 01m 10s]
val_accuracy: 0.9684153199195862

Best val_accuracy So Far: 0.9684153199195862
Total elapsed time: 05h 05m 04s
Best Hyperparameters: {'units': 128, 'dropout': 0.5, 'learning_rate': 0.001}
